# 알약 판별 EDA

In [2]:
# 노트북 상위 폴더 경로 추가
import sys
import os
sys.path.append(os.path.abspath(".."))

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import json
import os
import glob
from collections import Counter
import pandas as pd


# 유틸리티 함수 임포트
from src.utils import get_device, init_logger

In [4]:
logger = init_logger(name="healtheat_vision")  # 여기서만 프로젝트 이름을 지정
device = get_device()

logger.info(f"Using device: {device}")

2025-12-12 16:42:42 [INFO] (600008572.py:4) - Using device: mps


Apple MPS GPU detected.


In [5]:
from pathlib import Path
import json
import pprint

root = Path("../data")
ann_aihub_dir = root / "aihub_downloads"

if not ann_aihub_dir.exists():
    logger.error("경로가 존재하지 않습니다. 데이터 폴더를 확인하세요.")
else:
    logger.info("데이터 경로 확인 완료.")

2025-12-12 16:42:46 [INFO] (2300492895.py:11) - 데이터 경로 확인 완료.


In [6]:
from pathlib import Path
import json
import pprint

# 0) 테스트용 JSON 1개 잡기
json_paths = sorted(ann_aihub_dir.rglob("*.json"))
print("aihub_downloads 내 JSON 수:", len(json_paths))

test_path = json_paths[0]  # 일단 첫 파일
print("테스트 파일:", test_path)

data = json.loads(test_path.read_text())

print("\n[BEFORE] categories / annotations / images[0].dl_*")
pprint.pp(data["categories"])
pprint.pp(data["annotations"][:1])
pprint.pp({k: data["images"][0].get(k) for k in ["id", "dl_idx", "dl_name"]})

# 1) 변환(메모리에서만)
img0 = data["images"][0]
dl_idx = img0.get("dl_idx")
dl_name = img0.get("dl_name")

# dl_idx가 문자열이면 정수로 바꾸기 (category_id, categories.id는 int가 안전)
dl_idx_int = int(dl_idx) if dl_idx is not None else None

# categories를 dl 기준으로 단일 재구성
data["categories"] = [{
    "supercategory": "pill",
    "id": dl_idx_int,
    "name": dl_name
}]

# annotations의 category_id를 dl_idx로 교체
for ann in data.get("annotations", []):
    ann["category_id"] = dl_idx_int

print("\n[AFTER] categories / annotations / images[0].dl_*")
pprint.pp(data["categories"])
pprint.pp(data["annotations"][:1])
pprint.pp({k: data["images"][0].get(k) for k in ["id", "dl_idx", "dl_name"]})

aihub_downloads 내 JSON 수: 75093
테스트 파일: ../data/aihub_downloads/json_edited/raw_extracted/K-001900-010224-016551-031705_json/K-016551/K-001900-010224-016551-031705_0_2_0_2_70_000_200.json

[BEFORE] categories / annotations / images[0].dl_*
[{'supercategory': 'pill', 'id': 1, 'name': 'Drug'}]
[{'area': 128250,
  'iscrowd': 0,
  'bbox': [566, 131, 285, 450],
  'category_id': 1,
  'ignore': 0,
  'segmentation': [],
  'id': 1,
  'image_id': 1}]
{'id': 1, 'dl_idx': '16550', 'dl_name': '동아가바펜틴정 800mg'}

[AFTER] categories / annotations / images[0].dl_*
[{'supercategory': 'pill', 'id': 16550, 'name': '동아가바펜틴정 800mg'}]
[{'area': 128250,
  'iscrowd': 0,
  'bbox': [566, 131, 285, 450],
  'category_id': 16550,
  'ignore': 0,
  'segmentation': [],
  'id': 1,
  'image_id': 1}]
{'id': 1, 'dl_idx': '16550', 'dl_name': '동아가바펜틴정 800mg'}


In [8]:
from pathlib import Path
import json

src_root = ann_aihub_dir
json_paths = list(src_root.rglob("*.json"))

for src_path in json_paths:
    if "json_edited" in src_path.parts:
        continue

    try:
        _ = json.loads(src_path.read_text())
    except json.JSONDecodeError as e:
        print("❌ JSON 깨짐 파일:", src_path)
        print("에러:", e)

        # 에러 위치 근처 200자 미리보기
        txt = src_path.read_text()
        start = max(e.pos - 200, 0)
        end = min(e.pos + 200, len(txt))
        print("\n--- 에러 위치 근처 ---")
        print(txt[start:end])
        print("\n---------------------")
        break

❌ JSON 깨짐 파일: ../data/aihub_downloads/raw_extracted/K-003544-004543-016548-023203_json/K-023203/K-003544-004543-016548-023203_0_2_0_2_90_000_200.json
에러: Expecting ',' delimiter: line 62 column 4 (char 1604)

--- 에러 위치 근처 ---
de_front": "",
			"mark_code_back": "",
			"change_date": "20201210",
			"id": 1
		}
	],
	"type": "instances",
	"annotations": [
		{
			"area": 19043,
			"iscrowd": 0,
			"bbox": [655,853,139,137]
			"category_id": 1,
			"ignore": 0,
			"segmentation": [],
			"id": 1,
			"image_id": 1
		}
	],
	"categories": [
		{
			"supercategory": "pill",
			"id": 1,
			"name": "Drug"
		}
	]
}

---------------------


In [10]:
bad_encoding = []
bad_json = []

for src_path in json_paths:
    if "json_edited" in src_path.parts:
        continue

    try:
        text = src_path.read_text(encoding="utf-8", errors="replace")
    except Exception as e:
        bad_encoding.append((str(src_path), str(e)))
        continue

    try:
        _ = json.loads(text)
    except json.JSONDecodeError as e:
        bad_json.append((str(src_path), str(e)))
        continue

print("인코딩 문제 파일 수:", len(bad_encoding))
print("JSON 문법 깨짐 파일 수:", len(bad_json))

인코딩 문제 파일 수: 0
JSON 문법 깨짐 파일 수: 5


In [12]:
edited_paths = sorted(dst_root.rglob("*.json"))
print("edited JSON 수:", len(edited_paths))

sample = edited_paths[0]
d = json.loads(sample.read_text(encoding="utf-8"))

print(d["categories"])
print(d["annotations"][0]["category_id"], d["images"][0]["dl_idx"], d["images"][0]["dl_name"])

edited JSON 수: 81088
[{'supercategory': 'pill', 'id': 249, 'name': '마그밀정(수산화마그네슘)'}]
249 249 마그밀정(수산화마그네슘)
